# Participants Selection

In [8]:
# Import libraries
import numpy as np
import pandas as pd
import csv
from datetime import datetime

In [9]:
# File containing all types of codes:
dates = pd.read_csv("/Users/marinacamacho/Desktop/Master_I/Raw_Data/f.41280_ukbb.csv",delimiter="\,")
codes = pd.read_csv("/Users/marinacamacho/Desktop/Master_I/Raw_Data/f.41270_ukbb.csv",delimiter="\,")

In [10]:
del codes['Unnamed: 0']
del dates['Unnamed: 0']

In [11]:
Healthy = pd.DataFrame()
Healthy['eid'] = codes[pd.isna(codes['41270-0.0'])]['eid']

In [12]:
Healthy.to_csv(r'/Users/marinacamacho/Desktop/Master_I/Participants_Selection/Healthy_Participants.csv', index = False, header=True)

In [13]:
Depression = ['F32','F33']
CVD = ['F01','I20','I21','I22','I23','I24','I25','I50','I6']
Diabetes = ['E100','E101','E102','E103','E104','E105','E106','E107','E108','E109','E110',
            'E111','E112','E113','E114','E115','E116','E117','E118','E119','E121','E123',
            'E125','E128','E129','E130','E131','E132','E133','E134','E134','E135','E136',
            'E137','E138','E139','E140','E141','E142','E143','E144','E145','E146','E147','E148','E149']

In [14]:
codes_dis = [Depression,CVD,Diabetes]
names_dis = ['Depression','CVD','Diabetes']
dataframe_disease = {"eid":[], "code":[], "date":[]}

In [15]:
len(names_dis)

3

In [16]:
codes.shape, dates.shape

((502481, 224), (502481, 224))

In [17]:
# Iterate over codes and names for diseases
for x in range(0,len(codes_dis)):

    # Get disease codes and names for current iteration
    my_list = codes_dis[x]
    disease_name = names_dis[x]

    # Create a mask to filter rows that contain disease codes
    mask = codes.applymap(lambda x: x.startswith(tuple(my_list)) if isinstance(x, str) else False)
    # Apply the mask to get a filtered version of 'codes' dataframe
    codes_n = codes[mask.any(axis=1)]

    # Filter 'dates' dataframe to include only entries present in 'codes_n'
    dates_n = dates.loc[dates['eid'].isin(codes_n['eid'])].sort_values(by=['eid']).reset_index(drop=True)
    # Filter 'codes_n' similarly
    codes_n = codes_n.loc[codes_n['eid'].isin(dates_n['eid'])].sort_values(by=['eid']).reset_index(drop=True)

    # Save filtered dataframes to csv
    codes_n.to_csv(r'codes_n.csv', index = False, header=True)
    dates_n.to_csv(r'dates_n.csv', index = False, header=True)

    # Specify disease code prefixes
    code_prefixes = my_list

    # create an empty dictionary to store the information
    data = {}

    # Read the disease codes file
    with open("codes_n.csv", "r") as f:
        reader = csv.reader(f)
        headers = next(reader)  # save the header row

        for row in reader:
            eid = row[0]  # get the eid
            for i in range(1, len(row)):
                code = row[i]
                
                # If code starts with the specified prefixes, add it to the dictionary
                if code and any(code.startswith(prefix) for prefix in code_prefixes):
                    if eid not in data:
                        data[eid] = {}
                    data[eid][headers[i]] = {"code": disease_name} 
    
    # Get list of eids
    rows = list(pd.DataFrame(data).columns)
    dates_n = dates_n.set_index(['eid'])
    
    # Add date information for each eid and disease code
    for row in rows:
        for col in range(0,pd.DataFrame(data[row]).shape[1]):
            cols = list(data[row].keys())
            cols_r = [s.replace('41270', '41280') for s in cols]
            date = dates_n[cols_r[col]][int(row)]
            data[row][cols[col]]['date'] = date  
            
    # Transform data dictionary into a pandas dataframe
    df_data = pd.DataFrame(data)  
    
    # Find the earliest date for each disease for each eid
    for colum in range(0,df_data.shape[1]):   
        min_time = '2024-1-1'
        min_time = datetime.strptime(min_time, '%Y-%m-%d').date()
        eid=''
        for row in range(0,df_data.shape[0]):
            if str(df_data.values[row][colum]) != 'nan':
                date_string = df_data.values[row][colum]['date']
                date_object = datetime.strptime(date_string, '%Y-%m-%d').date()
                if min_time > date_object:
                    min_time = date_object
                    eid = df_data.columns[colum]

        # Store the earliest date along with corresponding eid and disease code            
        dataframe_disease.setdefault("eid", []).append(eid)
        dataframe_disease.setdefault("code", []).append(disease_name)
        dataframe_disease.setdefault("date", []).append(min_time)

In [18]:
pd.DataFrame(dataframe_disease['code']).value_counts()

CVD           73539
Diabetes      41296
Depression    28281
dtype: int64

In [19]:
diseases = pd.DataFrame(dataframe_disease)

In [20]:
# To check that this is working correctly we can explore the 
# results of the first eid in teh datafame

In [21]:
dates.iloc[14:15]

,eid,41280-0.0,41280-0.1,41280-0.2,41280-0.3,41280-0.4,41280-0.5,41280-0.6,41280-0.7,41280-0.8,...,41280-0.213,41280-0.214,41280-0.215,41280-0.216,41280-0.217,41280-0.218,41280-0.219,41280-0.220,41280-0.221,41280-0.222
14,1000159,2007-11-14,2008-06-19,2009-10-08,2010-04-28,2003-09-03,2014-10-06,2010-05-11,2008-12-11,2012-04-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
codes.iloc[14:15]

,eid,41270-0.0,41270-0.1,41270-0.2,41270-0.3,41270-0.4,41270-0.5,41270-0.6,41270-0.7,41270-0.8,...,41270-0.213,41270-0.214,41270-0.215,41270-0.216,41270-0.217,41270-0.218,41270-0.219,41270-0.220,41270-0.221,41270-0.222
14,1000159,D125,D128,F329,F410,F412,G473,I848,I849,J342,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
diseases.head(1)

,eid,code,date
0,1000159,Depression,2009-10-08


In [24]:
# Sort the dataframe
diseases = diseases.sort_values(by=['eid', 'date']).reset_index(drop=True); diseases

,eid,code,date
0,1000028,CVD,1997-12-12
1,1000087,CVD,2015-12-12
2,1000087,Diabetes,2015-12-12
3,1000159,Depression,2009-10-08
4,1000302,Diabetes,2011-06-15
...,...,...,...
143111,6024768,Depression,2019-10-01
143112,6024784,CVD,2011-05-12
143113,6024795,Depression,2019-08-28
143114,6024795,CVD,2019-08-28


In [25]:
diseases.to_csv(r'participants_diseases_raw.csv', index = False, header=True)

In [26]:
codes.shape

(502481, 224)

In [27]:
diseases.shape

(143116, 3)

In [28]:
no_diseases = codes[~codes['eid'].astype(str).isin(diseases['eid'].astype(str))].sort_values(by=['eid'])

In [29]:
pd.DataFrame(no_diseases['eid']).to_csv(r'no_diseases.csv', index = False, header=True)

In [30]:
no_diseases.shape

(387090, 224)

In [31]:
wide_diseases = diseases.pivot(index='eid', columns='code', values='date')
wide_diseases.reset_index(inplace=True)

In [32]:
wide_diseases.to_csv(r'wide_diseases_raw.csv', index = False, header=True)

In [33]:
wide_diseases

code,eid,CVD,Depression,Diabetes
0,1000028,1997-12-12,NaN,NaN
1,1000087,2015-12-12,NaN,2015-12-12
2,1000159,NaN,2009-10-08,NaN
3,1000302,NaN,NaN,2011-06-15
4,1000430,NaN,NaN,2016-07-11
...,...,...,...,...
115386,6024702,2020-11-21,2000-03-06,NaN
115387,6024753,NaN,2011-09-24,NaN
115388,6024768,NaN,2019-10-01,NaN
115389,6024784,2011-05-12,NaN,NaN


In [34]:
var_temp = {'eid': 'eid',
    '53-0.0': 'date_centre_0',
    '53-1.0': 'date_centre_1'}
date_centre_all = pd.read_csv('/Users/marinacamacho/Desktop/Master_I/Raw_Data/ukb46359.csv', usecols = var_temp.keys())
date_centre_all = date_centre_all.rename(columns = var_temp).sort_values(by=['eid']).reset_index(drop=True)

In [35]:
date_centre_all.to_csv(r'Date_Centre.csv', index = False, header=True)

In [36]:
date_centre_all

,eid,date_centre_0,date_centre_1
0,1000010,2010-02-25,NaN
1,1000028,2009-10-26,NaN
2,1000034,2009-05-13,NaN
3,1000045,2009-08-07,NaN
4,1000052,2009-01-05,NaN
...,...,...,...
502476,6024770,2009-09-12,NaN
502477,6024784,2008-10-23,NaN
502478,6024795,2008-04-22,NaN
502479,6024804,2009-07-07,NaN


In [37]:
date_centre = date_centre_all.loc[date_centre_all['eid'].astype(int).isin(wide_diseases['eid'].astype(int))].sort_values(by=['eid']).reset_index(drop=True)
wide_diseases['visit0'] = date_centre['date_centre_0']
wide_diseases['visit1'] = date_centre['date_centre_1']

date_centre.shape[0]==wide_diseases.index.shape[0]

True

In [38]:
# List of columns containing dates
date_columns = list(set(wide_diseases.columns).difference(['eid']))

# Convert elements in date columns to datetime objects
for col in date_columns:
    wide_diseases[col] = pd.to_datetime(wide_diseases[col])

In [39]:
wide_diseases

code,eid,CVD,Depression,Diabetes,visit0,visit1
0,1000028,1997-12-12,NaT,NaT,2009-10-26,NaT
1,1000087,2015-12-12,NaT,2015-12-12,2008-12-06,NaT
2,1000159,NaT,2009-10-08,NaT,2008-04-01,NaT
3,1000302,NaT,NaT,2011-06-15,2008-02-20,NaT
4,1000430,NaT,NaT,2016-07-11,2010-04-09,NaT
...,...,...,...,...,...,...
115386,6024702,2020-11-21,2000-03-06,NaT,2008-08-16,NaT
115387,6024753,NaT,2011-09-24,NaT,2007-12-15,NaT
115388,6024768,NaT,2019-10-01,NaT,2008-04-12,NaT
115389,6024784,2011-05-12,NaT,NaT,2008-10-23,NaT


In [40]:
wide_diseases.dropna(subset=['visit0']).to_csv(r'wide_diseases_dates1.csv', index = False, header=True)

In [41]:
wide_diseases.dropna(subset=['visit0']).shape

(115391, 6)

In [42]:
wide_diseases.dropna(subset=['visit0','visit1']).to_csv(r'wide_diseases_dates2.csv', index = False, header=True)

In [43]:
wide_diseases.dropna(subset=['visit0','visit1']).shape

(4169, 6)

In [44]:
wide_diseases.shape

(115391, 6)

In [179]:
#wide_diseases

In [191]:
#wide_diseases_visitD = wide_diseases[(wide_diseases['visit0'] <= wide_diseases['Depression'] - pd.DateOffset(years=year_diff)) | 
#                                    ((wide_diseases['visit0'] >= wide_diseases['CVD'] + pd.DateOffset(years=year_diff)) & not
#                                    (wide_diseases['visit0'] >= wide_diseases['Depression'] + pd.DateOffset(years=year_diff))) |
#                                    ((wide_diseases['visit0'] >= wide_diseases['Diabetes'] + pd.DateOffset(years=year_diff)) & not
#                                    (wide_diseases['visit0'] >= wide_diseases['Depression'] + pd.DateOffset(years=year_diff)))]

In [192]:
# Set minimum year difference between visit0 and other columns
year_diff = 1

# Select rows where visit0 is smaller or equal to min_date minus year_diff
condition1 = (wide_diseases['visit0'] <= wide_diseases['Depression'] - pd.DateOffset(years=year_diff))

condition2_a = (wide_diseases['visit0'] >= wide_diseases['CVD'] + pd.DateOffset(years=year_diff))
condition2_b = ~(wide_diseases['visit0'] >= wide_diseases['Depression'] + pd.DateOffset(years=year_diff))
condition2 = condition2_a & condition2_b

condition3_a = (wide_diseases['visit0'] >= wide_diseases['Diabetes'] + pd.DateOffset(years=year_diff))
condition3_b = ~(wide_diseases['visit0'] >= wide_diseases['Depression'] + pd.DateOffset(years=year_diff))
condition3 = condition3_a & condition3_b

wide_diseases_visitD = wide_diseases[condition1 | condition2 | condition3]


In [193]:
wide_diseases_visitD.to_csv(r'PREDICTIVE_VISIT_DEPRESSION.csv', index = False, header=True)

In [194]:
wide_diseases_visitD.shape

(47725, 6)

In [ ]:
#wide_diseases

In [155]:
# Set minimum year difference between visit0 and other columns
year_diff = 1

# Get the columns containing dates (excluding eid, visit0, and visit1)
date_columns = list(set(wide_diseases.columns).difference(['eid', 'visit0', 'visit1']))

# Compute the minimum date across the other columns for each row
max_date = wide_diseases[date_columns].max(axis=1)

# Select rows where visit0 is smaller or equal to min_date minus year_diff
wide_diseases_visit = wide_diseases[wide_diseases['visit0'] <= max_date - pd.DateOffset(years=year_diff)]

In [156]:
wide_diseases_visit.to_csv(r'PREDICTIVE_VISIT_ALL.csv', index = False, header=True)

In [157]:
wide_diseases_visit.shape

(83794, 6)

In [63]:
#wide_diseases

In [71]:
# Set minimum year difference between visit0 and other columns
year_diff = 1

# Get the columns containing dates (excluding eid, visit0, and visit1)
date_columns = list(set(wide_diseases.columns).difference(['eid', 'visit0', 'visit1']))

# Compute the minimum date across the other columns for each row
min_date = wide_diseases[date_columns].min(axis=1)

# Select rows where visit0 is smaller or equal to min_date minus year_diff
wide_diseases_visit0 = wide_diseases[wide_diseases['visit0'] <= min_date - pd.DateOffset(years=year_diff)]

In [72]:
wide_diseases_visit0.to_csv(r'PREDICTIVE_VISIT0_ALL.csv', index = False, header=True)

In [73]:
wide_diseases_visit0.shape

(74893, 6)

In [74]:
# Set minimum year difference between visit0 and other columns
year_diff = 1

# Get the columns containing dates (excluding eid, visit0, and visit1)
date_columns = list(set(wide_diseases.columns).difference(['eid', 'visit0', 'visit1']))

# Compute the minimum date across the other columns for each row
min_date = wide_diseases[date_columns].min(axis=1)

# Select rows where visit0 is smaller or equal to min_date minus year_diff
wide_diseases_visit1 = wide_diseases[wide_diseases['visit1'] <= min_date - pd.DateOffset(years=year_diff)]

In [75]:
wide_diseases_visit1.to_csv(r'PREDICTIVE_VISIT1_ALL.csv', index = False, header=True)

In [76]:
wide_diseases_visit1.shape

(1985, 6)

In [77]:
df = wide_diseases_visit0.replace({pd.Timestamp.min: 0, np.datetime64('NaT'): 0, True: 1, False: 0})

In [78]:
df = df.astype(bool).astype(int)

In [79]:
df['eid'] = wide_diseases_visit0['eid']

In [85]:
df.shape

(74893, 6)

In [83]:
df.head()

code,eid,CVD,Depression,Diabetes,visit0,visit1
1,1000087,1,0,1,1,0
2,1000159,0,1,0,1,0
3,1000302,0,0,1,1,0
4,1000430,0,0,1,1,0
5,1000465,1,0,0,1,0


In [84]:
df.to_csv(r'Prediction_Binary.csv', index = False, header=True)

# Comorbidity

In [86]:
# Set minimum year difference between visit0 and other columns
year_diff = 1

# Get the columns containing dates (excluding eid, visit0, and visit1)
date_columns = list(set(wide_diseases.columns).difference(['eid', 'visit0', 'visit1']))

# Compute the minimum date across the other columns for each row
min_date = wide_diseases[date_columns].min(axis=1)

#### CVD-Depression

In [87]:
# Select rows where visit0 is smaller or equal to min_date minus year_diff
wide_diseases_d = wide_diseases[wide_diseases['visit0'] <= (wide_diseases['Depression'] - pd.DateOffset(years=year_diff))]
wide_diseases_h = wide_diseases[wide_diseases['CVD'] <= wide_diseases['visit0']]
wide_diseases_d = wide_diseases_d[wide_diseases_d['eid'].isin(wide_diseases_h['eid'])]
wide_diseases_h = wide_diseases_h[~wide_diseases_h['eid'].isin(wide_diseases_d['eid'])]
time_d = wide_diseases_d['Depression'] - wide_diseases_d['visit0']
wide_diseases_h['Y']=0
wide_diseases_d['Y']=1

CVDDep_hd = pd.concat([wide_diseases_h,wide_diseases_d],axis=0).sort_values(by=['eid']).reset_index(drop=True)
CVDDep_hd.to_csv(r'CVDDep_hd_newUKBB.csv', index = False, header=True)
CVDDep_hd['Y'].value_counts()

0    22396
1     1782
Name: Y, dtype: int64

In [88]:
# Find the minimum and maximum values of the time_d column
time_d_min = time_d.min()
time_d_max = time_d.max()

# Transform the minimum and maximum values to years
days_per_year = 365.25
time_d_min_years = time_d_min.total_seconds() / (days_per_year * 86400)
time_d_max_years = time_d_max.total_seconds() / (days_per_year * 86400)

print(time_d_min_years, time_d_max_years)

1.002053388090349 13.00479123887748


#### Depression-CVD

In [89]:
# Select rows where visit0 is smaller or equal to min_date minus year_diff
wide_diseases_d = wide_diseases[wide_diseases['visit0'] <= wide_diseases['CVD'] - pd.DateOffset(years=year_diff)]
wide_diseases_h = wide_diseases[wide_diseases['Depression'] <= wide_diseases['visit0']]
wide_diseases_d = wide_diseases_d[wide_diseases_d['eid'].isin(wide_diseases_h['eid'])]
wide_diseases_h = wide_diseases_h[~wide_diseases_h['eid'].isin(wide_diseases_d['eid'])]
time_d = wide_diseases_d['CVD'] - wide_diseases_d['visit0']
wide_diseases_h['Y']=0
wide_diseases_d['Y']=1

DepCVD_hd = pd.concat([wide_diseases_h,wide_diseases_d],axis=0).sort_values(by=['eid']).reset_index(drop=True)
DepCVD_hd.to_csv(r'DepCVD_hd_newUKBB.csv', index = False, header=True)
DepCVD_hd['Y'].value_counts()

0    4031
1     705
Name: Y, dtype: int64

In [90]:
# Find the minimum and maximum values of the time_d column
time_d_min = time_d.min()
time_d_max = time_d.max()

# Transform the minimum and maximum values to years
days_per_year = 365.25
time_d_min_years = time_d_min.total_seconds() / (days_per_year * 86400)
time_d_max_years = time_d_max.total_seconds() / (days_per_year * 86400)

print(time_d_min_years, time_d_max_years)

1.0047912388774811 13.494866529774127


#### Diabetes-CVD

In [91]:
# Select rows where visit0 is smaller or equal to min_date minus year_diff
wide_diseases_d = wide_diseases[wide_diseases['visit0'] <= wide_diseases['CVD'] - pd.DateOffset(years=year_diff)]
wide_diseases_h = wide_diseases[wide_diseases['Diabetes'] <= wide_diseases['visit0']]
wide_diseases_d = wide_diseases_d[wide_diseases_d['eid'].isin(wide_diseases_h['eid'])]
wide_diseases_h = wide_diseases_h[~wide_diseases_h['eid'].isin(wide_diseases_d['eid'])]
time_d = wide_diseases_d['CVD'] - wide_diseases_d['visit0']
wide_diseases_h['Y']=0
wide_diseases_d['Y']=1

DiaCVD_hd = pd.concat([wide_diseases_h,wide_diseases_d],axis=0).sort_values(by=['eid']).reset_index(drop=True)
DiaCVD_hd.to_csv(r'DiaCVD_hd_newUKBB.csv', index = False, header=True)
DiaCVD_hd['Y'].value_counts()

0    9024
1    2087
Name: Y, dtype: int64

In [92]:
#

In [93]:
# Find the minimum and maximum values of the time_d column
time_d_min = time_d.min()
time_d_max = time_d.max()

# Transform the minimum and maximum values to years
days_per_year = 365.25
time_d_min_years = time_d_min.total_seconds() / (days_per_year * 86400)
time_d_max_years = time_d_max.total_seconds() / (days_per_year * 86400)

print(time_d_min_years, time_d_max_years)

1.0047912388774811 13.757700205338809


#### CVD-Diabetes

In [94]:
# Select rows where visit0 is smaller or equal to min_date minus year_diff
wide_diseases_d = wide_diseases[wide_diseases['visit0'] <= wide_diseases['Diabetes'] - pd.DateOffset(years=year_diff)]
wide_diseases_h = wide_diseases[wide_diseases['CVD'] <= wide_diseases['visit0']]
wide_diseases_d = wide_diseases_d[wide_diseases_d['eid'].isin(wide_diseases_h['eid'])]
wide_diseases_h = wide_diseases_h[~wide_diseases_h['eid'].isin(wide_diseases_d['eid'])]
time_d = wide_diseases_d['Diabetes'] - wide_diseases_d['visit0']
wide_diseases_h['Y']=0
wide_diseases_d['Y']=1

CVDDia_hd = pd.concat([wide_diseases_h,wide_diseases_d],axis=0).sort_values(by=['eid']).reset_index(drop=True)
CVDDia_hd.to_csv(r'CVDDia_hd_newUKBB.csv', index = False, header=True)
CVDDia_hd['Y'].value_counts()

0    21139
1     3039
Name: Y, dtype: int64

In [95]:
#

In [96]:
# Find the minimum and maximum values of the time_d column
time_d_min = time_d.min()
time_d_max = time_d.max()

# Transform the minimum and maximum values to years
days_per_year = 365.25
time_d_min_years = time_d_min.total_seconds() / (days_per_year * 86400)
time_d_max_years = time_d_max.total_seconds() / (days_per_year * 86400)

print(time_d_min_years, time_d_max_years)

0.999315537303217 14.343600273785079


#### Depression-Diabetes

In [97]:
# Select rows where visit0 is smaller or equal to min_date minus year_diff
wide_diseases_d = wide_diseases[wide_diseases['visit0'] <= wide_diseases['Diabetes'] - pd.DateOffset(years=year_diff)]
wide_diseases_h = wide_diseases[wide_diseases['Depression'] <= wide_diseases['visit0']]
wide_diseases_d = wide_diseases_d[wide_diseases_d['eid'].isin(wide_diseases_h['eid'])]
wide_diseases_h = wide_diseases_h[~wide_diseases_h['eid'].isin(wide_diseases_d['eid'])]
time_d = wide_diseases_d['Diabetes'] - wide_diseases_d['visit0']
wide_diseases_h['Y']=0
wide_diseases_d['Y']=1

DepDia_hd = pd.concat([wide_diseases_h,wide_diseases_d],axis=0).sort_values(by=['eid']).reset_index(drop=True)
DepDia_hd.to_csv(r'DepDia_hd_newUKBB.csv', index = False, header=True)
DepDia_hd['Y'].value_counts()

0    4230
1     506
Name: Y, dtype: int64

In [98]:
# Find the minimum and maximum values of the time_d column
time_d_min = time_d.min()
time_d_max = time_d.max()

# Transform the minimum and maximum values to years
days_per_year = 365.25
time_d_min_years = time_d_min.total_seconds() / (days_per_year * 86400)
time_d_max_years = time_d_max.total_seconds() / (days_per_year * 86400)

print(time_d_min_years, time_d_max_years)

1.002053388090349 13.141683778234086


#### Diabetes-Depression

In [99]:
# Select rows where visit0 is smaller or equal to min_date minus year_diff
wide_diseases_d = wide_diseases[wide_diseases['visit0'] <= wide_diseases['Depression'] - pd.DateOffset(years=year_diff)]
wide_diseases_h = wide_diseases[wide_diseases['Diabetes'] <= wide_diseases['visit0']]
wide_diseases_d = wide_diseases_d[wide_diseases_d['eid'].isin(wide_diseases_h['eid'])]
wide_diseases_h = wide_diseases_h[~wide_diseases_h['eid'].isin(wide_diseases_d['eid'])]
time_d = wide_diseases_d['Depression'] - wide_diseases_d['visit0']
wide_diseases_h['Y']=0
wide_diseases_d['Y']=1

DiaDep_hd = pd.concat([wide_diseases_h,wide_diseases_d],axis=0).sort_values(by=['eid']).reset_index(drop=True)
DiaDep_hd.to_csv(r'DiaDep_hd_newUKBB.csv', index = False, header=True)
DiaDep_hd['Y'].value_counts()

0    10063
1     1048
Name: Y, dtype: int64

In [100]:
DiaDep_hd

code,eid,CVD,Depression,Diabetes,visit0,visit1,Y
0,1001032,NaT,NaT,2004-02-21,2009-09-04,NaT,0
1,1001565,2003-10-14,NaT,2007-04-30,2007-11-15,NaT,0
2,1001573,2002-04-14,NaT,2002-04-14,2010-01-08,NaT,0
3,1001683,2003-03-20,2019-12-03,2007-09-03,2009-02-12,NaT,1
4,1001800,NaT,NaT,2008-05-06,2010-06-10,NaT,0
...,...,...,...,...,...,...,...
11106,6023624,NaT,NaT,1998-05-29,2008-09-22,NaT,0
11107,6024057,NaT,NaT,2008-11-25,2009-11-28,NaT,0
11108,6024156,2008-10-23,2019-09-05,2007-11-24,2008-11-22,NaT,1
11109,6024390,NaT,NaT,1996-04-22,2007-11-08,NaT,0


In [101]:
# Find the minimum and maximum values of the time_d column
time_d_min = time_d.min()
time_d_max = time_d.max()

# Transform the minimum and maximum values to years
days_per_year = 365.25
time_d_min_years = time_d_min.total_seconds() / (days_per_year * 86400)
time_d_max_years = time_d_max.total_seconds() / (days_per_year * 86400)

print(time_d_min_years, time_d_max_years)

1.0321697467488022 13.36892539356605


# Trajectories

In [ ]:
from pandas.tseries.offsets import DateOffset

In [135]:
df = wide_diseases.dropna(subset=['visit0','visit1']).sort_values(by=['eid']).reset_index(drop=True)

# convert your columns to datetime if they aren't already
for column in ['CVD', 'Depression', 'Diabetes', 'visit0', 'visit1']:
    df[column] = pd.to_datetime(df[column])

# define your new columns
df['time0'] = df.apply(lambda row: sum((pd.notnull(row[condition])) & 
                                       (row[condition] < row['visit0'] - DateOffset(years=1))
                                       for condition in ['CVD', 'Depression', 'Diabetes']), axis=1)

df['time1'] = df.apply(lambda row: sum((pd.notnull(row[condition])) & 
                                       (row[condition] > row['visit0'] + DateOffset(years=1)) &
                                       (row[condition] < row['visit1'] - DateOffset(years=1))
                                       for condition in ['CVD', 'Depression', 'Diabetes']), axis=1)

df['time2'] = df.apply(lambda row: sum((pd.notnull(row[condition])) &
                                       (row[condition] > row['visit1'] + DateOffset(years=1))
                                       for condition in ['CVD', 'Depression', 'Diabetes']), axis=1)

In [136]:
filtered_df = df[(df['time0'] <= 1) & (df['time1'] <= 1) & (df['time2'] <= 1)]

In [137]:
df.shape, filtered_df.shape

((4169, 9), (3807, 9))

In [138]:
filtered_df

code,eid,CVD,Depression,Diabetes,visit0,visit1,time0,time1,time2
0,1004849,NaT,2019-08-29,1999-10-14,2008-06-20,2013-02-04,1,0,1
1,1006558,NaT,2014-09-01,NaT,2007-12-17,2013-01-30,0,0,1
2,1008230,NaT,1998-05-26,NaT,2007-06-25,2012-08-31,1,0,0
3,1008381,NaT,NaT,2011-01-10,2008-06-09,2012-11-01,0,1,0
4,1011321,NaT,2013-04-03,NaT,2008-03-03,2013-05-25,0,0,0
...,...,...,...,...,...,...,...,...,...
4163,6018451,NaT,NaT,2010-05-24,2008-02-14,2012-08-21,0,1,0
4164,6019794,2018-03-01,NaT,NaT,2007-12-11,2013-05-05,0,0,1
4165,6019978,2004-11-05,NaT,NaT,2009-04-24,2013-01-23,1,0,0
4166,6020459,NaT,NaT,2009-05-20,2008-09-01,2013-05-12,0,0,0


In [139]:
df = wide_diseases.dropna(subset=['visit0','visit1']).sort_values(by=['eid']).reset_index(drop=True)

# convert your columns to datetime if they aren't already
for column in ['CVD', 'Depression', 'Diabetes', 'visit0', 'visit1']:
    df[column] = pd.to_datetime(df[column])

# define your new columns
df['time0'] = df.apply(lambda row: [condition for condition in ['CVD', 'Depression', 'Diabetes'] 
                                    if (pd.notnull(row[condition]) & 
                                    (row[condition] < row['visit0'] - pd.DateOffset(years=1)))], axis=1)

df['time1'] = df.apply(lambda row: [condition for condition in ['CVD', 'Depression', 'Diabetes'] 
                                    if (pd.notnull(row[condition]) & 
                                    (row[condition] > row['visit0'] + pd.DateOffset(years=1)) &
                                    (row[condition] < row['visit1'] - pd.DateOffset(years=1)))], axis=1)

df['time2'] = df.apply(lambda row: [condition for condition in ['CVD', 'Depression', 'Diabetes'] 
                                    if (pd.notnull(row[condition]) &
                                    (row[condition] > row['visit1'] + pd.DateOffset(years=1)))], axis=1)


In [140]:
df

code,eid,CVD,Depression,Diabetes,visit0,visit1,time0,time1,time2
0,1004849,NaT,2019-08-29,1999-10-14,2008-06-20,2013-02-04,[Diabetes],[],[Depression]
1,1006558,NaT,2014-09-01,NaT,2007-12-17,2013-01-30,[],[],[Depression]
2,1008230,NaT,1998-05-26,NaT,2007-06-25,2012-08-31,[Depression],[],[]
3,1008381,NaT,NaT,2011-01-10,2008-06-09,2012-11-01,[],[Diabetes],[]
4,1011321,NaT,2013-04-03,NaT,2008-03-03,2013-05-25,[],[],[]
...,...,...,...,...,...,...,...,...,...
4164,6019794,2018-03-01,NaT,NaT,2007-12-11,2013-05-05,[],[],[CVD]
4165,6019978,2004-11-05,NaT,NaT,2009-04-24,2013-01-23,[CVD],[],[]
4166,6020459,NaT,NaT,2009-05-20,2008-09-01,2013-05-12,[],[],[]
4167,6021437,2019-04-26,2019-04-26,NaT,2008-06-13,2012-11-08,[],[],"[CVD, Depression]"


In [141]:
df = wide_diseases.dropna(subset=['visit0','visit1']).sort_values(by=['eid']).reset_index(drop=True)

# convert your columns to datetime if they aren't already
for column in ['CVD', 'Depression', 'Diabetes', 'visit0', 'visit1']:
    df[column] = pd.to_datetime(df[column])

# define your new columns
df['time0'] = df.apply(lambda row: [row[condition] for condition in ['CVD', 'Depression', 'Diabetes'] 
                                    if (pd.notnull(row[condition]) & 
                                    (row[condition] < row['visit0'] - pd.DateOffset(years=1)))], axis=1)

df['time1'] = df.apply(lambda row: [row[condition] for condition in ['CVD', 'Depression', 'Diabetes'] 
                                    if (pd.notnull(row[condition]) & 
                                    (row[condition] > row['visit0'] + pd.DateOffset(years=1)) &
                                    (row[condition] < row['visit1'] - pd.DateOffset(years=1)))], axis=1)

df['time2'] = df.apply(lambda row: [row[condition] for condition in ['CVD', 'Depression', 'Diabetes'] 
                                    if (pd.notnull(row[condition]) &
                                    (row[condition] > row['visit1'] + pd.DateOffset(years=1)))], axis=1)


In [142]:
df

code,eid,CVD,Depression,Diabetes,visit0,visit1,time0,time1,time2
0,1004849,NaT,2019-08-29,1999-10-14,2008-06-20,2013-02-04,[1999-10-14 00:00:00],[],[2019-08-29 00:00:00]
1,1006558,NaT,2014-09-01,NaT,2007-12-17,2013-01-30,[],[],[2014-09-01 00:00:00]
2,1008230,NaT,1998-05-26,NaT,2007-06-25,2012-08-31,[1998-05-26 00:00:00],[],[]
3,1008381,NaT,NaT,2011-01-10,2008-06-09,2012-11-01,[],[2011-01-10 00:00:00],[]
4,1011321,NaT,2013-04-03,NaT,2008-03-03,2013-05-25,[],[],[]
...,...,...,...,...,...,...,...,...,...
4164,6019794,2018-03-01,NaT,NaT,2007-12-11,2013-05-05,[],[],[2018-03-01 00:00:00]
4165,6019978,2004-11-05,NaT,NaT,2009-04-24,2013-01-23,[2004-11-05 00:00:00],[],[]
4166,6020459,NaT,NaT,2009-05-20,2008-09-01,2013-05-12,[],[],[]
4167,6021437,2019-04-26,2019-04-26,NaT,2008-06-13,2012-11-08,[],[],"[2019-04-26 00:00:00, 2019-04-26 00:00:00]"
